In [19]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [30]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Adrar,MR,2020-04-06 21:25:25,20.50,-10.07,82.31,13,0,11.21
1,1,Astana,KZ,2020-04-06 21:22:48,51.18,71.45,37.40,86,100,11.18
2,2,Bredasdorp,ZA,2020-04-06 21:23:24,-34.53,20.04,55.99,62,15,11.41
3,3,Ushuaia,AR,2020-04-06 21:21:18,-54.80,-68.30,57.20,50,75,16.11
4,4,Hermanus,ZA,2020-04-06 21:25:26,-34.42,19.23,55.99,71,16,5.99


In [31]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [37]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [39]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [40]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [41]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [42]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [43]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Adrar,MR,2020-04-06 21:25:25,20.50,-10.07,82.31,13,0,11.21
6,6,Saint-Pierre,RE,2020-04-06 21:25:26,-21.34,55.48,77.00,73,20,6.93
9,9,Bonthe,SL,2020-04-06 21:25:27,7.53,-12.51,81.59,85,77,6.35
15,15,Kant,IN,2020-04-06 21:25:28,27.80,79.80,75.22,24,0,7.99
18,18,Conde,BR,2020-04-06 21:25:28,-7.26,-34.91,82.40,83,20,9.17
23,23,Axim,GH,2020-04-06 21:25:29,4.87,-2.24,82.35,81,0,5.44
26,26,Souillac,MU,2020-04-06 21:25:29,-20.52,57.52,78.01,83,40,8.05
27,27,Vila Velha,BR,2020-04-06 21:24:47,-20.33,-40.29,78.80,78,75,10.29
33,33,Francisco Sa,BR,2020-04-06 21:25:30,-16.48,-43.49,86.00,55,20,4.70
35,35,Mahajanga,MG,2020-04-06 21:15:38,-15.72,46.32,78.80,88,20,1.23


In [44]:
preferred_cities_df.count()

City_ID       195
City          195
Country       195
Date          195
Lat           195
Lng           195
Max Temp      195
Humidity      195
Cloudiness    195
Wind Speed    195
dtype: int64

In [47]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

In [50]:
hotel_df["Hotel Name"] = ""

In [52]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Adrar,MR,82.31,20.50,-10.07,
6,Saint-Pierre,RE,77.00,-21.34,55.48,
9,Bonthe,SL,81.59,7.53,-12.51,
15,Kant,IN,75.22,27.80,79.80,
18,Conde,BR,82.40,-7.26,-34.91,
23,Axim,GH,82.35,4.87,-2.24,
26,Souillac,MU,78.01,-20.52,57.52,
27,Vila Velha,BR,78.80,-20.33,-40.29,
33,Francisco Sa,BR,86.00,-16.48,-43.49,
35,Mahajanga,MG,78.80,-15.72,46.32,


In [53]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}